# Directv Programming Guide - Data Cleaning & Validation process

This notebook aim is to clean and validate data scrapped from the Directv programming guide using a Scrapy spider

Author [@santteegt](https://santteegt.github.io)

In [1]:
import pandas as pd
import json
import requests
import hashlib

In [2]:
scrapy_project = 'directvscrapper'
spider = 'directv'
job_id = ''

## Scheduling Directv spider on Scrapy server

**NOTE**: Skip the cell below If spider has alredy been scheduled

In [8]:
rs = requests.post('http://localhost:6800/schedule.json', data={'project': scrapy_project, 'spider': spider})
json_rs = json.loads(rs.text)
print(json_rs)
if json_rs['status'] == 'error':
    print('ERROR -> %s' % json_rs['message'])
else:
    job_id = json_rs['jobid']
    print('Spider has been scheduled. Job id -> %s' % job_id)

{'jobid': '5a57a8dcccad11e897600242ac110002', 'node_name': '7f9006259b53', 'status': 'ok'}
Spider has been scheduled. Job id -> 5a57a8dcccad11e897600242ac110002


## Data Cleaning & Validation

**IMPORTANT**: After the scraping process finished, run the following cells

In [9]:
scrapy_data_dir = 'data/items/directvscrapper/directv/'

json_file = '093df40c760e11e793300242ac110002.jl' if not job_id else job_id + '.jl'
json_guide = 'scrapped_data/programming_guide.json'
json_desc = 'scrapped_data/programming_desc.json'

In [10]:
! mkdir scrapped_data

mkdir: scrapped_data: File exists


In [11]:
programming_guide = []
programming_desc = []

with open(scrapy_data_dir + json_file, encoding='utf-8') as f:
    while True:
        json_line = f.readline()
        if not json_line:
            break
        
        jl = json.loads(json_line)
        [programming_guide.append(jl) if 'show_id' in jl else programming_desc.append(jl)]


with open(json_guide, 'w') as f_guide:
    json.dump(programming_guide, f_guide)
#     [f_guide.write(json.dumps(j) + '\n') for j in programming_guide]
#     f_guide.write(json.dumps(programming_guide))
    f_guide.close()
    
with open(json_desc, 'w') as f_desc:
    json.dump(programming_desc, f_desc)
#     [f_desc.write(json.dumps(j) + '\n') for j in programming_desc]
#     f_desc.write(json.dumps(programming_desc))
    f_desc.close()
    

print(len(programming_guide))
print(len(programming_desc))

35369
15685


In [85]:
# df_guide = pd.read_csv(json_guide)
df_guide = pd.io.json.json_normalize(programming_guide)
df_guide['query_date'] = pd.to_datetime(df_guide['query_date'])
print(df_guide.shape)
df_guide.head()

(35369, 8)


,channel_name,channel_number,day,query_date,show_id,start_time,time_length,title
0,INFO,100,Miércoles,2018-10-10 12:00:00,281247488289,10:00,360,Canal Informativo DIRECTV
1,101,101,Miércoles,2018-10-10 12:00:00,4140246043023,10:00,360,Canal 101
2,AINF,128,Miércoles,2018-10-10 12:00:00,128247668099,12:00,30,Flawless V1
3,TLEC,177,Miércoles,2018-10-10 12:00:00,433247626775,12:00,30,El Gran Cacao
4,TA,180,Miércoles,2018-10-10 12:00:00,180247762324,11:45,40,Doraemón


In [13]:
df_guide['channel_name'].unique()

array(['INFO', '101', 'AINF', 'TLEC', 'TA', 'TC', 'EV', 'GAMA', 'RTS',
       'ECTV', 'TLRM', 'UNO', 'ORO', 'TVCE', 'ODTV', 'FOX', 'WTV', 'A&E',
       'SONY', 'AXN', 'AMC', 'FOXL', 'TNTS', 'A3SE', 'CMDY', 'TBS', 'FX',
       'UNI', 'SYFY', 'TRU', 'HOLATV', 'E!', 'ID', 'GLTZ', 'PASN', 'CEST',
       'NVLA', 'LIFE', 'DH&H', 'CHIC', 'TLMD', 'GOUR', 'FOOD', 'MTV',
       'QMMI', 'HTV', 'DTV', 'FFAM', 'HBOF', 'TOON', 'NICK', 'DSNY',
       'DSJR', 'DSXD', 'NGKD', 'ZMOO', 'KIDS', 'BOOM', 'EWTN', 'ORBE21',
       'ENLC', 'CINE', 'PPV', 'SMS', 'PBTV', 'VNUS', 'XTSY', 'HUST', 'TNT',
       'GLDN', 'TCM', 'FXMV', 'CNCL', 'STUN', 'CMAX', 'PRMT', 'SUND', 'CL',
       'DPEL', 'EDGE', 'SPAC', 'ISAT', 'NEXT', 'HBOE', 'HBO2', 'HBOP',
       'HBOS', 'PLUS', 'MAXP', 'MAXE', 'MAXU', 'PRME', 'FPSE', 'FPSO',
       'FOXF', 'FOXC', 'FOXM', 'FCLA', 'FOXA', 'FCIN'], dtype=object)

## Omitting non national TV stations

In [111]:
df_nat_guide = df_guide[~df_guide.channel_name.isin(
    ['INFO', '101', 'AINF', 'DTV', 'SMS', 'PBTV', 'VNUS', 'XTSY', 'HUST', 'NEXT'])].copy()
df_nat_guide.shape

(30987, 8)

In [112]:
df = df_nat_guide.drop_duplicates(['channel_number', 'day', 'show_id', 'start_time']).copy()
print(df.shape)
df.head()

(21370, 8)


,channel_name,channel_number,day,query_date,show_id,start_time,time_length,title
3,TLEC,177,Miércoles,2018-10-10 12:00:00,433247626775,12:00,30,El Gran Cacao
4,TA,180,Miércoles,2018-10-10 12:00:00,180247762324,11:45,40,Doraemón
5,TC,181,Miércoles,2018-10-10 12:00:00,181247625874,11:00,85,Los Hijos de Don Juan
6,EV,182,Miércoles,2018-10-10 12:00:00,182247623871,10:30,150,En Contacto
7,GAMA,183,Miércoles,2018-10-10 12:00:00,183248008947,12:00,30,Cocinando con Campana


## Generating hash id from title

In [113]:
df['id'] = df['title'].apply(lambda x: hashlib.sha1(x.encode('UTF-8')).hexdigest()[:10])
df.head()

,channel_name,channel_number,day,query_date,show_id,start_time,time_length,title,id
3,TLEC,177,Miércoles,2018-10-10 12:00:00,433247626775,12:00,30,El Gran Cacao,6e262b999f
4,TA,180,Miércoles,2018-10-10 12:00:00,180247762324,11:45,40,Doraemón,35fa610b01
5,TC,181,Miércoles,2018-10-10 12:00:00,181247625874,11:00,85,Los Hijos de Don Juan,71ea6d42d3
6,EV,182,Miércoles,2018-10-10 12:00:00,182247623871,10:30,150,En Contacto,f6d3df555f
7,GAMA,183,Miércoles,2018-10-10 12:00:00,183248008947,12:00,30,Cocinando con Campana,4f9cc090be


## Loading Programming description

In [114]:
df_desc = pd.io.json.json_normalize(programming_desc)
df_desc.columns = ['description', 'show_id']
print(df_desc.shape)
df_desc.head()

(15685, 2)


,description,show_id
0,Un hombre y una mujer que crecieron juntos a p...,227247625150
1,Tres hermanos juran vengarse de la familia que...,226247625296
2,Una historia de amor intenso entre una joven h...,225247624192
3,"La peculiar conductora Carmen Jara, con su per...",224247624440
4,"T02 | E06 Asesinan a Angela Moss, una enfermer...",223247627487


In [115]:
df_desc = df_desc[df_desc['show_id'].isin(df['show_id'].values)]
df_desc.shape

(14142, 2)

In [116]:
# guide_index = df.set_index('show_id')
guide_index = df.drop_duplicates('show_id').set_index('show_id')
guide_index.shape

(14152, 8)

### Appending generated id from programmming guide records

In [117]:
df_desc['id'] = df_desc['show_id'].apply(lambda x: guide_index.loc[str(x), 'id'])
df_desc.head()

,description,show_id,id
0,Un hombre y una mujer que crecieron juntos a p...,227247625150,d55a7bf26f
1,Tres hermanos juran vengarse de la familia que...,226247625296,05effdcbb2
2,Una historia de amor intenso entre una joven h...,225247624192,ac50038a04
3,"La peculiar conductora Carmen Jara, con su per...",224247624440,5b2c0c3541
4,"T02 | E06 Asesinan a Angela Moss, una enfermer...",223247627487,ba2dcb7806


### Removing records with no `show_id`

In [118]:
df = df[df['show_id'].str.len() > 0]
df.shape

(19569, 9)

In [119]:
temp = df[~df['show_id'].isin(df_desc['show_id'].values)]

In [120]:
if temp.shape[0] > 0:
    print('WARNING: There are {} records with missing descriptions. Now removing them...'.format(temp.shape[0]))
    df = df[df['show_id'].isin(df_desc['show_id'].values)]
df.shape

(19558, 9)

In [121]:
temp = df[~df['show_id'].isin(df_desc['show_id'].values)]

In [124]:
assert(temp.shape[0] == 0)
df.to_csv('scrapped_data/programming_guide.csv', index=False)
print('Toral Programming Guide records: %s' % df.shape[0])
print('Unique programming records: %d' % guide_index.shape[0])
# df_desc_filtered = df_desc[df_desc['show_id'].isin(df['show_id'].values)]
# df_desc_filtered.to_csv('scrapped_data/programming_desc.csv', index=False)
df_desc.to_csv('scrapped_data/programming_desc.csv', index=False)
# print('Toral Programming description records: %s' % df_desc_filtered.shape[0])
print('Toral Programming description records: %s' % df_desc.shape[0])
print('All programming data have been scrapped successfully')
print('Directv programming Guide from %s to %s' % (df['query_date'].min(), df['query_date'].max()))

Toral Programming Guide records: 19558
Unique programming records: 14152
Toral Programming description records: 14142
All programming data have been scrapped correctly
Directv programming Guide from 2018-10-10 12:00:00 to 2018-10-16 23:30:00


**NOTE** Last cell shows that Directv is not usingn a unique identifier for each show

In [125]:
df_desc.sort_values(['description'])

,description,show_id,id
13305,"""La Casa del Pop"" invita a los artistas más de...",269247676697,afdf6a807b
2295,"""La Casa del Pop"" invita a los artistas más de...",269247707990,afdf6a807b
13471,"""La Casa del Pop"" invita a los artistas más de...",269247676693,afdf6a807b
2671,"""Los Hijos del Dragón"", un grupo terrorista de...",526247712762,95e85bd847
11537,"""Maricruz"" es una joven dulce e inocente que v...",227247706854,9537784dd4
9751,"""Maricruz"" es una joven dulce e inocente que v...",227247738109,9537784dd4
6543,"""Maricruz"" es una joven dulce e inocente que v...",227247806651,9537784dd4
13133,"""Maricruz"" es una joven dulce e inocente que v...",227247676173,9537784dd4
15165,"""Maricruz"" es una joven dulce e inocente que v...",227247625158,9537784dd4
7743,"""Maricruz"" es una joven dulce e inocente que v...",227247771605,9537784dd4
